# ITCS 6162: Data Mining - Programming Assignment

**In this assignment, you will explore data analysis, recommendation algorithms, and graph-based techniques using the MovieLens dataset. Your tasks will range from basic data exploration to advanced recommendation models, including:**
- Data manipulation with pandas
- User-item collaborative filtering
- Similarity-based recommendation models
- A Pixie-inspired Graph-based recommendation using adjacency lists with weighted random walks (without using NetworkX)


#### **Dataset Files:**
- **`u.data`**: User-movie ratings (`user_id  movie_id  rating  timestamp`)
- **`u.item`**: Movie metadata (`movie_id | title | release date | IMDB_website`)
- **`u.user`**: User demographics (`user_id | age | gender | occupation | zip_code`)

## **Part 1: Exploring and Cleaning Data**

### Inspecting the Dataset Format

The dataset is not in a traditional CSV format. To examine its structure, use the following shell command to display the first 10 lines of the file:

```sh
!head <file_name>


**In the cells given below. Write the code to read the files.**

### Import Libraries  
We import required Python libraries such as `pandas`, `numpy`, `random`, `sklearn`, and others needed for data manipulation and similarity calculations.
These will be used for data loading, manipulation, and building recommendation logic.

In [68]:
import random
import pandas as pd
import numpy as np 
from tabulate import tabulate
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

### Define Path for Ratings Data  
The `ratings_path` variable stores the file path to the `u.data` file, which contains user-movie interactions and ratings.

In [69]:
ratings_path = "u.data"

###  Read Ratings Data  
We read the ratings file using `read_csv` with tab separator and assign meaningful column names: `user_id`, `movie_id`, `rating`, `timestamp`.

In [70]:
# u.data
ratings = pd.read_csv(ratings_path, sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])

###  Define Path for Movie Metadata  
We define the path to the `u.item` file which contains movie information.

In [71]:
items_path = "u.item"

###  Read Movies Data  
Load the movie metadata file using `|` separator and select only the required columns: `movie_id`, `title`, `release_date`.

In [72]:
# u.item
movies = pd.read_csv(items_path, sep='|', encoding='latin-1', usecols=[0, 1, 2], names=["movie_id", "title", "release_date"])

###  Define Path for User Information  
The `users_path` variable holds the path to the user metadata file `u.user`.

In [73]:
users_path = "u.user"

###  Load the Users Data

We load the `u.user` dataset into a DataFrame named `users`.  
This dataset contains demographic information such as user ID, age, gender, and occupation.

Separator used: `|` 
Columns selected: index 0 to 3
Column names: `"user_id"`, `"age"`, `"gender"`, `"occupation"`

This information can later be used for user profiling or demographic analysis.

In [74]:
# u.user
users = pd.read_csv(users_path, sep="|", usecols=[0, 1, 2, 3], names=["user_id", "age", "gender", "occupation"])

#### Loading the Dataset with Pandas

Use **pandas** to load the dataset into a DataFrame for analysis. Follow these steps:  

1. Import the necessary library: `pandas`.  
2. Use `pd.read_csv()` (or an appropriate function) to read the dataset file.  
3. Ensure the dataset is loaded with the correct delimiter (e.g., `','`, `'\t'`,`'|'` , or another separator if needed).  
4. Select and display the first few rows using `.head()`.

Ensure that:  

- The `ratings` dataset is read from `"u.data"` using tab (`'\t'`) as a separator and column names (`"user_id"`, `"movie_id"`, `"rating"` and `"timestamp"`).  
- The `movies` dataset is read from `"u.item"` using `'|'` as a separator, use columns (`0`, `1`, `2`), encoding (`"latin-1"`) and name the columns (`movie_id`, `title`, and `release_date`).  
- The `users` dataset is read from `"u.user"` using `'|'` as a separator, use columns (`0`, `1`, `2`, `3`) and name the columns (`user_id`, `age`, `gender`, and `occupation`).

### Bonus Task: Save the DataFrames as `.csv` Files

As a bonus task, we save the three DataFrames — `ratings`, `movies`, and `users` — into separate `.csv` files.

This is used to:
Persisting the cleaned and structured data
Reusing the data for other analysis or machine learning tasks
Sharing the datasets with others

We use the `to_csv()` method provided by pandas and set `index=False` to exclude the DataFrame index from the saved file.


### Load and Verify All DataFrames

We read the datasets using the correct delimiters to confirm all the data was loaded succeddfully into the DataFrame : `"ratings"`, `"movies"`, `"users"`.

In [75]:
# ratings
ratings = pd.read_csv(ratings_path, sep='\t', names=["user_id", "movie_id", "rating", "timestamp"])

In [76]:
# movies
movies = pd.read_csv(items_path, sep='|', encoding='latin-1', usecols=[0, 1, 2], names=["movie_id", "title", "release_date"])

In [77]:
# users
users = pd.read_csv(users_path, sep='|', usecols=[0, 1, 2, 3], names=["user_id", "age", "gender", "occupation"])

**Note:** As a **Bonus** task save the `ratings`, `movies` and `users` dataframe created into a `.csv` file format. <br>
**Hint:** Use the `to_csv()` function in pandas to save these DataFrames as CSV files.

### Saving DataFrames to CSV

We save the `ratings`, `movies`, and `users` DataFrames to separate `.csv` files using `to_csv()`, with `index=False` to skip row numbers.

In [78]:
# ratings
ratings.to_csv("ratings.csv", index=False)

In [79]:
# movies
movies.to_csv("movies.csv", index=False)

In [80]:
# users
users.to_csv("users.csv", index=False)

**Display the first 10 rows of each file.**

### Display First 10 Rows

Use `.head(10)` to view the top 10 entries from each DataFrame for a quick check.

In [81]:
# ratings
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


### Displayed first 10 rows from the `movies` DataFrame to preview movie IDs, titles, and release dates.

In [82]:
# movies
movies.head(10)

,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995
6,7,Twelve Monkeys (1995),01-Jan-1995
7,8,Babe (1995),01-Jan-1995
8,9,Dead Man Walking (1995),01-Jan-1995
9,10,Richard III (1995),22-Jan-1996


### Displayed the first 10 rows of the `users` DataFrame showing user ID, age, gender, and occupation.

In [83]:
# users
users.head(10)

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
5,6,42,M,executive
6,7,57,M,administrator
7,8,36,M,administrator
8,9,29,M,student
9,10,53,M,lawyer


### Data Cleaning and Exploration with Pandas  

After loading the dataset, it’s important to clean and explore the data to ensure consistency and accuracy. Below are key **pandas** functions for cleaning and understanding the dataset.

#### 1. Handle Missing Values  
- `df.dropna()` – Removes rows with missing values.  
- `df.fillna(value)` – Fills missing values with a specified value.  

#### 2. Remove Duplicates  
- `df.drop_duplicates()` – Drops duplicate rows from the dataset.  

#### 3. Handle Incorrect Data Types  
- `df.astype(dtype)` – Converts columns to the appropriate data type.  

#### 4. Filter Outliers (if applicable)  
- `df[df['column_name'] > threshold]` – Filters rows based on a condition.  

#### 5. Rename Columns (if needed)  
- `df.rename(columns={'old_name': 'new_name'})` – Renames columns for clarity.  

#### 6. Reset Index  
- `df.reset_index(drop=True, inplace=True)` – Resets the index after cleaning.  

### Data Exploration Functions  

To better understand the dataset, use these **pandas** functions:  

- `df.shape` – Returns the number of rows and columns in the dataset.  
- `df.nunique()` – Displays the number of unique values in each column.  
- `df['column_name'].unique()` – Returns unique values in a specific column.  

**Example Usage in Pandas:**  
```python
import pandas as pd

# Load dataset
df = pd.read_csv("your_file.csv")

# Drop missing values
df_cleaned = df.dropna()

# Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Convert 'timestamp' column to datetime format
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

# Display dataset shape
print("Dataset shape:", df_cleaned.shape)

# Display number of unique values in each column
print("Unique values per column:\n", df_cleaned.nunique())

# Display unique movie IDs
print("Unique movie IDs:", df_cleaned['movie_id'].unique()[:10])  # Show first 10 unique movie IDs


**Note:** The functions mentioned above are some of the widely used **pandas** functions for data cleaning and exploration. However, it is not necessary that all of these functions will be required in the exercises below. Use them as needed based on the dataset and the specific tasks.

**Convert Timestamps into Readable dates.**

### Data Preprocessing

Converted `timestamp` in the `ratings` DataFrame to readable datetime format using `pd.to_datetime`.

In [85]:
# ratings
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head(10)

,user_id,movie_id,rating,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16
5,298,474,4,1998-01-07 14:20:06
6,115,265,2,1997-12-03 17:51:28
7,253,465,5,1998-04-03 18:34:27
8,305,451,3,1998-02-01 09:20:17
9,6,86,3,1997-12-31 21:16:53


**Check for Missing Values**

### `ratings`: No missing values.

In [87]:
# ratings
ratings.isnull().sum()


user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

### `movies`: 1 missing value found in the `release_date` column.

In [88]:
# movies
movies.isnull().sum()

movie_id        0
title           0
release_date    1
dtype: int64

### `users`: No missing values detected in any column (`user_id`, `age`, `gender`, `occupation`).


In [89]:
# users
users.isnull().sum()

user_id       0
age           0
gender        0
occupation    0
dtype: int64

**Print the total number of users, movies, and ratings.**

**Total Counts:**

In [90]:
print(f"Total Users: {users['user_id'].nunique()}")
print(f"Total Movies: {movies['movie_id'].nunique()}")
print(f"Total Ratings: {len(ratings)}")

Total Users: 943
Total Movies: 1682
Total Ratings: 100000


## **Part 2: Collaborative Filtering-Based Recommendation**

### **Create a User-Item Matrix**

#### Instructions for Creating a User-Movie Rating Matrix

In this exercise, you will create a user-movie rating matrix using **pandas**. This matrix will represent the ratings that users have given to different movies.

1. **Dataset Overview**:  
   The dataset has already been loaded. It includes the following key columns:
   - `user_id`: The ID of the user.
   - `movie_id`: The ID of the movie.
   - `ratings`: The rating the user gave to the movie.

2. **Create the User-Movie Rating Matrix**:  
   Use the **`pivot()`** function in **pandas** to reshape the data. Your goal is to create a matrix where:
   - Each **row** represents a **user**.
   - Each **column** represents a **movie**.
   - Each **cell** contains the **rating** that the user has given to the movie.

   Specify the following parameters for the `pivot()` function:
   - **`index`**: The `user_id` column (this will define the rows).
   - **`columns`**: The `movie_id` column (this will define the columns).
   - **`values`**: The `rating` column (this will fill the matrix with ratings).

3. **Inspect the Matrix**:  
   After creating the matrix, examine the first few rows of the resulting matrix to ensure it has been constructed correctly.

4. **Handle Missing Values**:  
   It's likely that some users have not rated every movie, resulting in `NaN` values in the matrix. You will need to handle these missing values. Consider the following options:
   - **Fill with 0**: If you wish to represent missing ratings as zeros (indicating no rating).
   - **Fill with the average rating**: Alternatively, replace missing values with the average rating for each movie.

### Create the User-Movie Rating Matrix

In this step, we are transforming the raw ratings DataFrame into a **user-item matrix** using the `pivot()` function from pandas.

- Each **row** corresponds to a user (`user_id`)
- Each **column** corresponds to a movie (`movie_id`)
- Each **cell** contains the **rating** given by that user to that movie
This matrix will help us apply collaborative filtering techniques by comparing users and their movie preferences.


**Create the user-movie rating matrix using the `pivot()` function.**

In [91]:
user_movie_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating')

### Display the Matrix

We preview the top few rows of the resulting `user_movie_matrix` using `.head()` to verify that the transformation was successful.

Note:
- The presence of `NaN` values indicates that a user hasn't rated those particular movies.
- The matrix has a shape of (number of users) × (number of movies), with each cell showing the corresponding rating.

**Display the matrix to verify the transformation.**

In [92]:
user_movie_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Handle missing values by filling NaNs with column-wise averages:

- Since many users have not rated every movie, the matrix contains missing values (`NaN`). 
- To handle this, we replace each missing value in a column with the average rating for that movie across all users. 
- This ensures that all cells are filled and the matrix is usable for similarity calculations.


In [93]:
user_movie_matrix = user_movie_matrix.apply(lambda col: col.fillna(col.mean()), axis=0)

### Round the matrix values for clarity:

- After filling missing values, we round the ratings to one decimal place. 
- This step is primarily for better readability and doesn't affect the recommendation calculations significantly.


In [94]:
user_movie_matrix = user_movie_matrix.round(1)
user_movie_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
2,4.0,3.2,3.0,3.6,3.3,3.6,3.8,4.0,3.9,2.0,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
3,3.9,3.2,3.0,3.6,3.3,3.6,3.8,4.0,3.9,3.8,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
4,3.9,3.2,3.0,3.6,3.3,3.6,3.8,4.0,3.9,3.8,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0
5,4.0,3.0,3.0,3.6,3.3,3.6,3.8,4.0,3.9,3.8,...,3.0,4.0,3.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0


### **User-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement a **user-based collaborative filtering** movie recommendation system using the **Movie dataset**. The goal is to recommend movies to a user based on the preferences of similar users.

##### **Step 1: Import Required Libraries**
Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity
```

##### **Step 2: Compute User-User Similarity**
- We will use **cosine similarity** to measure how similar each pair of users is based on their movie ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.

##### **Instructions:**
1. Fill missing values with `0` using `.fillna(0)`.
2. Compute similarity using `cosine_similarity()`.
3. Convert the result into a **Pandas DataFrame**, with users as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
```

##### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies_for_user(user_id, num=5)` to recommend movies for a given user.

##### **Function Inputs:**
- `user_id`: The target user for whom we need recommendations.
- `num`: The number of movies to recommend (default is 5).

##### **Function Steps:**
1. Find **similar users**:
   - Retrieve the similarity scores for the given `user_id`.
   - Sort them in **descending** order (highest similarity first).
   - Exclude the user themselves.
   
2. Get the **movie ratings** from these similar users.

3. Compute the **average rating** for each movie based on these users' preferences.

4. Sort the movies in **descending order** based on the computed average ratings.

5. Retrieve the **top `num` recommended movies**.

6. Map **movie IDs** to their **titles** using the `movies` DataFrame.

7. Return the results as a **Pandas DataFrame** with rankings.

##### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'Ranking': range(1, num+1),
    'Movie Name': movie_names     
})
result_df.set_index('Ranking', inplace=True)
```

#### **Example: User-Based Collaborative Filtering**
```python
recommend_movies_for_user(10, num = 5)
```
**Output:**
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | In the Company of Men (1997)   |
| 2       | Misérables, Les (1995)         |
| 3       | Thin Blue Line, The (1988)     |
| 4       | Braindead (1992)               |
| 5       | Boys, Les (1997)               |


###  Function: `recommend_movies_for_user(user_id, num=5)`

This function implements a **user-based collaborative filtering recommendation system** using **cosine similarity**.

**Inputs:**
- `user_id` — the ID of the user to recommend movies for.
- `num` — number of movie recommendations to return (default is 5).

**What it does:**
1. Identifies all movies the target user has not rated.
2. Computes similarity between the target user and all other users using cosine similarity.
3. Predicts ratings for unseen movies by averaging ratings from similar users, weighted by similarity scores.
4. Sorts these movies by predicted rating and selects the top N.
5. Converts movie IDs to movie names and returns a nicely formatted DataFrame of recommendations.

**Returns:**  
A DataFrame with columns `Ranking` and `Movie Name`, sorted by predicted preference.


In [95]:
# Code the function here
def recommend_movies_for_user(user_id, num=5):
    user_rated_movies = ratings[ratings['user_id'] == user_id]['movie_id']
    unrated_movie_ids = movies[~movies['movie_id'].isin(user_rated_movies)]['movie_id']
    unrated_movie_ids = unrated_movie_ids.reset_index(drop=True)
    user_movie_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
    user_similarity = cosine_similarity(user_movie_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

    user_similarity_scores = user_similarity_df.loc[user_id]
    weighted_ratings = ratings.pivot(index='user_id', columns='movie_id', values='rating')
    weighted_ratings = weighted_ratings.T.dot(user_similarity_scores) / user_similarity_scores.sum()

    common_movie_ids = weighted_ratings.index.intersection(unrated_movie_ids)
    recommended_ratings = weighted_ratings.loc[common_movie_ids]

    recommended_movies = recommended_ratings.sort_values(ascending=False).head(num)

    recommended_movie_names = movies[movies['movie_id'].isin(recommended_movies.index)][['movie_id', 'title']]
    recommended_movie_names = recommended_movie_names.set_index('movie_id').loc[recommended_movies.index]['title']
    recommended_movie_names = recommended_movie_names.dropna()

    result_df = pd.DataFrame({
        'Ranking': range(1, len(recommended_movie_names) + 1),
        'Movie Name': recommended_movie_names.values
    }).set_index('Ranking')

    return result_df

### Step 4: Generate and Display Movie Recommendations

- Now that the `recommend_movies_for_user()` function is defined, we can generate personalized movie recommendations for a specific user.  

- The table below shows the top 5 recommended movies for `user_id = 10`, formatted using the `tabulate` library with a GitHub-style table layout for better readability.


In [96]:
result = recommend_movies_for_user(10, num=5)
print(tabulate(result, headers='keys', tablefmt='github'))

|   Ranking | Movie Name                                           |
|-----------|------------------------------------------------------|
|         1 | GoldenEye (1995)                                     |
|         2 | Four Rooms (1995)                                    |
|         3 | Copycat (1995)                                       |
|         4 | Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) |
|         5 | Babe (1995)                                          |


### **Item-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement an **item-based collaborative filtering** recommendation system using the **Movie dataset**. The goal is to recommend movies similar to a given movie based on user rating patterns.

#### **Step 1: Import Required Libraries**
Although we have done this part already in the previous task but just to emphasize the importance reiterrating this part.

Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing item similarity
```

#### **Step 2: Compute Item-Item Similarity**
- We will use **cosine similarity** to measure how similar each pair of movies is based on their user ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.
- Unlike user-based filtering, we need to **transpose** (`.T`) the `user_movie_matrix` because we want similarity between movies (columns) instead of users (rows).

##### **Instructions:**
1. Transpose the user-movie matrix using `.T` to make movies the rows.
2. Fill missing values with `0` using `.fillna(0)`.
3. Compute similarity using `cosine_similarity()`.
4. Convert the result into a **Pandas DataFrame**, with movies as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
item_similarity = cosine_similarity(user_movie_matrix.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)
```

#### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies(movie_name, num=5)` to recommend movies similar to a given movie.

##### **Function Inputs:**
- `movie_name`: The target movie for which we need recommendations.
- `num`: The number of similar movies to recommend (default is 5).

##### **Function Steps:**
1. Find the **movie_id** corresponding to the given `movie_name` in the `movies` DataFrame.
2. If the movie is not found, return an appropriate message.
3. Extract the **similarity scores** for this movie from `item_sim_df`.
4. Sort the movies in **descending order** based on similarity (excluding the movie itself).
5. Retrieve the **top `num` similar movies**.
6. Map **movie IDs** to their **titles** using the `movies` DataFrame.
7. Return the results as a **Pandas DataFrame** with rankings.

#### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'ranking': range(1, num+1),
    'movie_name': movie_names
})
result_df.set_index('ranking', inplace=True)
```

#### **Example: Item-Based Collaborative Filtering**
```python
recommend_movies("Jurassic Park (1993)", num=5)
```
**Output:**
```
| Ranking | Movie Name                               |
|---------|------------------------------------------|
| 1       | Top Gun (1986)                           |
| 2       | Empire Strikes Back, The (1980)          |
| 3       | Raiders of the Lost Ark (1981)           |
| 4       | Indiana Jones and the Last Crusade (1989)|
| 5       | Speed (1994)                             |


###  Recommend Movies Using Title Similarity (Item-Based Filtering)

This section implements a content-based recommender system using cosine similarity over movie titles.

####  Steps Explained:

1. **Load the movie data**:
   - Read the dataset from `movies.csv`.

2. **Vectorize movie titles**:
   - Use `CountVectorizer` to convert movie titles into vector representations (bag-of-words model).

3. **Compute pairwise similarity**:
   - Apply `cosine_similarity()` on the vectorized title matrix to measure title-based similarity between movies.

4. **Create a similarity DataFrame**:
   - Store the cosine similarity matrix in a DataFrame `item_sim_df` with movie IDs as both rows and columns.

5. **Define the recommendation function**:  
   `recommend_movies(movie_name, num=5)` recommends the top `num` most similar movie titles.

####  Function Logic:

- **Check title existence**: If the movie title doesn’t exist, return an error message.
- **Get movie ID**: Retrieve the movie ID for the input title.
- **Fetch similarity scores**: Get all similarity scores for this movie from `item_sim_df`.
- **Sort and select top N**: Drop the movie itself and sort remaining movies by similarity.
- **Map IDs to names**: Use `movies` DataFrame to convert movie IDs to titles.
- **Return nicely formatted result**: Output is printed as a GitHub-style markdown table.



In [97]:
# Code the function here

movies = pd.read_csv("movies.csv")
vectorizer = CountVectorizer().fit_transform(movies['title'])
similarity_matrix = cosine_similarity(vectorizer)
item_sim_df = pd.DataFrame(similarity_matrix, index=movies['movie_id'], columns=movies['movie_id'])

def recommend_movies(movie_name, num=5):
    if movie_name not in movies['title'].values:
        return f"Movie '{movie_name}' not found in dataset."

    movie_id = movies[movies['title'] == movie_name]['movie_id'].values[0]
    
    if movie_id not in item_sim_df.index:
        return f"Movie ID {movie_id} not found in similarity matrix."
    
    similarity_scores = item_sim_df.loc[movie_id]
    similar_movie_ids = similarity_scores.drop(movie_id).sort_values(ascending=False).head(num).index
    movie_names = movies.set_index('movie_id').loc[similar_movie_ids]['title'].tolist()

    result_df = pd.DataFrame({'Ranking': range(1, num+1), 'Movie Name': movie_names})
    print(result_df.to_markdown(index=False, tablefmt="github"))


- In the cell below, we call the recommend_movies() function with:

In [98]:
recommend_movies("Jurassic Park (1993)", num=5)

|   Ranking | Movie Name                            |
|-----------|---------------------------------------|
|         1 | Lost World: Jurassic Park, The (1997) |
|         2 | Robocop 3 (1993)                      |
|         3 | Fearless (1993)                       |
|         4 | Fausto (1993)                         |
|         5 | Shadowlands (1993)                    |


## **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm)**

### **Adjacency List**

#### **Objective**
In this task, you will preprocess the Movie dataset and construct a **graph representation** where:
- **Users** are connected to the movies they have rated.
- **Movies** are connected to users who have rated them.
  
This graph structure will help in exploring **user-movie relationships** for recommendations.

#### **Step 1: Merge Ratings with Movie Titles**
Since we have **movie IDs** in the ratings dataset but need human-readable movie titles, we will:
1. Merge the `ratings` DataFrame with the `movies` DataFrame using the `'movie_id'` column.
2. This allows each rating to be associated with a **movie title**.

#### **Hint:**
Use the following Pandas operation to merge:
```python
ratings = ratings.merge(movies, on='movie_id')
```


#### **Step 2: Aggregate Ratings**
Since multiple users may rate the same movie multiple times, we:
1. Group the dataset by `['user_id', 'movie_id', 'title']`.
2. Compute the **mean rating** for each movie by each user.
3. Reset the index to ensure we maintain a clean DataFrame structure.

#### **Hint:**  
Use `groupby()` and `mean()` as follows:
```python
ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()
```

#### **Step 3: Normalize Ratings**
Since different users have different rating biases, we normalize ratings by:
1. **Computing each user's mean rating**.
2. **Subtracting the mean rating** from each individual rating.

#### **Instructions:**
- Use `groupby('user_id')` to group ratings by users.
- Apply `transform(lambda x: x - x.mean())` to adjust ratings.

#### **Hint:**  
Normalize ratings using:
```python
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())
```
This ensures each user’s ratings are centered around zero, making similarity calculations fairer.

#### **Step 4: Construct the Graph Representation**
We represent the user-movie interactions as an **undirected graph** using an **adjacency list**:
- Each **user** is a node connected to movies they rated.
- Each **movie** is a node connected to users who rated it.

#### **Graph Construction Steps:**
1. Initialize an empty dictionary `graph = {}`.
2. Iterate through the **ratings dataset**.
3. For each `user_id` and `movie_id` pair:
   - Add the movie to the user’s set of connections.
   - Add the user to the movie’s set of connections.

#### **Hint:**  
The following code builds the graph:

```python
graph = {}
for _, row in ratings.iterrows():
    user, movie = row['user_id'], row['movie_id']
    if user not in graph:
        graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
    graph[user].add(movie)
    graph[movie].add(user)
```

This results in a **bipartite graph**, where:
- **Users** are connected to multiple movies.
- **Movies** are connected to multiple users.

#### **Step 5: Understanding the Graph**
- **Nodes** in the graph represent **users and movies**.
- **Edges** exist between a user and a movie **if the user has rated the movie**.
- This structure allows us to find **users with similar movie tastes** and **movies frequently watched together**.

#### **Exploring the Graph**
- **Find a user’s rated movies:**  
  ```python
  user_id = 1
  print(graph[user_id])  # Movies rated by user 1
  ```

- **Find users who rated a movie:**  
  ```python
  movie_id = 50
  print(graph[movie_id])  # Users who rated movie 50
  ```


- The dataset is loaded from `ratings.csv` and `movies.csv`, and merged using the `movie_id`.
- Ratings are grouped by user and movie to compute average ratings.
- Each user's ratings are normalized by subtracting their mean rating (mean centering).
- A bidirectional graph is built:
  - Users are connected to the movies they rated.
  - Movies are connected to the users who rated them.
- The graph is stored in a Python dictionary using sets for efficient lookups.
- The output displays:
  - All movies rated by a given user.
  - All users who rated a specific movie.

In [61]:
# Code the function here
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")
ratings = ratings.merge(movies, on='movie_id')

ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())

graph = {}
for _, row in ratings.iterrows():
    user = row['user_id']
    movie = row['movie_id']
    
    if user not in graph:
        graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
        
    graph[user].add(movie)
    graph[movie].add(user)

user_id = 1
print(f"Movies rated by user {user_id}: {graph[user_id]}")

movie_id = 50
print(f"Users who rated movie {movie_id}: {graph[movie_id]}")

Movies rated by user 1: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217

### **Implement Weighted Random Walks**

#### **Random Walk-Based Movie Recommendation System (Weighted Pixie)**

#### **Objective**
In this task, you will implement a **random-walk-based recommendation algorithm** using the **Weighted Pixie** method. This technique uses a **user-movie bipartite graph** to recommend movies by simulating a random walk from a given user or movie.

#### **Step 1: Import Required Libraries**
Make sure you have the necessary libraries:

```python
import random  # For random walks
import pandas as pd  # For handling data
```

#### **Step 2: Implement the Random Walk Algorithm**
Your task is to **simulate a random walk** from a given starting point in the **bipartite user-movie graph**.

##### **Hints for Implementation**
- Start from **either a user or a movie**.
- At each step, **randomly move** to a connected node.
- Keep track of **how many times each movie is visited**.
- After completing the walk, **rank movies by visit count**.

#### **Step 3: Implement User-Based Recommendation**
**Hints:**
- Check if the `user_id` exists in the `graph`.
- Start a loop that runs for `walk_length` steps.
- Randomly pick a **connected node** (user or movie).
- Track how many times each **movie** is visited.
- Sort movies by visit frequency and return the **top N**.

#### **Step 4: Implement Movie-Based Recommendation**
**Hints:**
- Find the `movie_id` corresponding to the given `movie_name`.
- Ensure the movie exists in the `graph`.
- Start a random walk from that movie.
- Follow the same **tracking and ranking** process as the user-based version.

**Note:**  
**Your task:** Implement a function `weighted_pixie_recommend(user_id, walk_length=15, num=5)` or `weighted_pixie_recommend(movie_name, walk_length=15, num=5)`.  
**Implement either Step 3 or Step 4.**

#### **Step 5: Running Your Recommendation System**
Once your function is implemented, test it by calling:

##### **Example: User-Based Recommendation**
```python
weighted_pixie_recommend(1, walk_length=15, num=5)
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | My Own Private Idaho (1991)   |
| 2       | Aladdin (1992)                |
| 3       | 12 Angry Men (1957)           |
| 4       | Happy Gilmore (1996)          |
| 5       | Copycat (1995)                |


##### **Example: Movie-Based Recommendation**
```python
weighted_pixie_recommend("Jurassic Park (1993)", walk_length=10, num=5)
```
| Ranking | Movie Name                           |
|---------|-------------------------------------|
| 1       | Rear Window (1954)                 |
| 2       | Great Dictator, The (1940)         |
| 3       | Field of Dreams (1989)             |
| 4       | Casablanca (1942)                  |
| 5       | Nightmare Before Christmas, The (1993) |


#### **Step 6: Understanding the Results**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

Each movie is ranked based on **how frequently it was visited** during the walk.

#### **Experiment with Different Parameters**
- Try different **`walk_length`** values and observe how it changes recommendations.
- Adjust the number of recommended movies (`num`).

###  Weighted Pixie Random Walk Recommendation Function

This cell implements a movie recommendation algorithm based on a **random walk over a bipartite user-movie graph**, inspired by the Pixie algorithm used by Pinterest.

- It first builds **bidirectional mappings** between movie IDs and titles, and between users and the movies they've rated.
- The core function `weighted_pixie_recommend()`:
  - Accepts a movie title as input and resolves its ID.
  - Performs **100 random walks** of specified `walk_length` starting from the movie.
  - Each walk alternates between users and movies:
    - From a movie → randomly pick a user who rated it.
    - From a user → randomly pick another movie they rated.
  - The algorithm counts how many times each movie node is visited.
- Finally, it selects the **top N most visited movies** and returns them in a ranked DataFrame.


In [127]:
# Code the function here
movie_id_col = 'movie_id'   
user_id_col = 'user_id'     
title_col = 'title'

movie_id_to_title = {}
title_to_id = {}

for _, row in movies.iterrows():
    movie_id_to_title[row[movie_id_col]] = row[title_col]
    title_to_id[row[title_col]] = row[movie_id_col]

user_to_movies = defaultdict(set)
movie_to_users = defaultdict(set)

for _, row in ratings.iterrows():
    user_id = row[user_id_col]
    movie_id = row[movie_id_col]
    user_to_movies[user_id].add(movie_id)
    movie_to_users[movie_id].add(user_id)

def weighted_pixie_recommend(start_title, walk_length=15, num=5):
    start_movie_id = title_to_id.get(start_title) 
    if start_movie_id is None:
        try:
            start_movie_id = int(start_title)
            if start_movie_id not in movie_id_to_title:
                raise ValueError(f"Movie ID '{start_title}' not found in dataset.")
        except ValueError:
            raise ValueError(f"Movie '{start_title}' not found in dataset.")

    visit_counts = Counter()

    for _ in range(100):  
        current_node = start_movie_id
        for _ in range(walk_length):
            if isinstance(current_node, (int, np.integer)):
                users = list(movie_to_users.get(current_node, []))
                if not users:
                    break
                current_node = random.choice(users)
            else:
                movies_watched = list(user_to_movies.get(current_node, []))
                if not movies_watched:
                    break
                current_node = random.choice(movies_watched)

            if isinstance(current_node, (int, np.integer)) and current_node != start_movie_id:
                visit_counts[current_node] += 1

    top_movies = visit_counts.most_common(num)
    top_titles = [movie_id_to_title[mid] for mid, _ in top_movies]

    result_df = pd.DataFrame({
        'Ranking': range(1, len(top_titles) + 1),
        'Movie Name': top_titles
    })

    return result_df

-  Recommendations for similar movies to *Jurassic Park (1993)* using Weighted Pixie random walks:


In [128]:
weighted_pixie_recommend("Jurassic Park (1993)", walk_length=15, num=5)

,Ranking,Movie Name
0,1,Mighty Aphrodite (1995)
1,2,Twelve Monkeys (1995)
2,3,Mission: Impossible (1996)
3,4,Leaving Las Vegas (1995)
4,5,My Own Private Idaho (1991)


-  Recommendations for similar movies to *Dead Man Walking (1995)* using Weighted Pixie random walks:


In [132]:
weighted_pixie_recommend("Dead Man Walking (1995)", walk_length=15, num=5)

,Ranking,Movie Name
0,1,Mighty Aphrodite (1995)
1,2,Star Trek V: The Final Frontier (1989)
2,3,Twelve Monkeys (1995)
3,4,Speechless (1994)
4,5,Stand by Me (1986)


- Recommendations for movie ID "100" using 10 results and a walk length of 15


In [134]:
weighted_pixie_recommend("100", walk_length=15, num=10)

,Ranking,Movie Name
0,1,Mighty Aphrodite (1995)
1,2,Mission: Impossible (1996)
2,3,Leaving Las Vegas (1995)
3,4,Twelve Monkeys (1995)
4,5,Lawnmower Man 2: Beyond Cyberspace (1996)
5,6,Old Yeller (1957)
6,7,My Own Private Idaho (1991)
7,8,Star Trek V: The Final Frontier (1989)
8,9,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
9,10,True Lies (1994)


---

## **Submission Requirements:**

To successfully complete this assignment, ensure that you submit the following:


### **1. Jupyter Notebook Submission**
- Submit a **fully completed Jupyter Notebook** that includes:
  - **All implemented recommendation functions** (user-based, item-based, and random walk-based recommendations).
  - **Code explanations** in markdown cells to describe each step.
  - **Results and insights** from running your recommendation models.


### **2. Explanation of Pixie-Inspired Algorithms (3-5 Paragraphs)**
- Write a **detailed explanation** of **Pixie-inspired random walk algorithms** used for recommendations.
- Your explanation should cover:
  - What **Pixie-inspired recommendation systems** are.
  - How **random walks** help in identifying relevant recommendations.
  - Any real-world applications of such algorithms in industry.


### **3. Report for the Submitted Notebook**
Your report should be structured as follows:

#### **Title: Movie Recommendation System Report**

#### **1. Introduction**
- Briefly introduce **movie recommendation systems** and why they are important.
- Explain the **different approaches used** (user-based, item-based, random-walk).

#### **2. Dataset Description**
- Describe the **MovieLens 100K dataset**:
  - Number of users, movies, and ratings.
  - What features were used.
  - Any preprocessing performed.

#### **3. Methodology**
- Explain the three recommendation techniques implemented:
  - **User-based collaborative filtering** (how user similarity was calculated).
  - **Item-based collaborative filtering** (how item similarity was determined).
  - **Random-walk-based Pixie algorithm** (why graph-based approaches are effective).
  
#### **4. Implementation Details**
- Discuss the steps taken to build the functions.
- Describe how the **adjacency list graph** was created.
- Explain how **random walks** were performed and how visited movies were ranked.

#### **5. Results and Evaluation**
- Present **example outputs** from each recommendation approach.
- Compare the different methods in terms of accuracy and usefulness.
- Discuss any **limitations** in the implementation.

#### **6. Conclusion**
- Summarize the key takeaways from the project.
- Discuss potential improvements (e.g., **hybrid models, additional features**).
- Suggest real-world applications of the methods used.

### **Submission Instructions**

- Submit `.zip` file consisting of Jupyter Notebook and all the datafiles (provided) and the ones saved [i.e. `users.csv`, `movies.csv` and `ratings.csv`]. Also, include the Report and Pixie Algorithm explanation document.
- [`Bonus 10 Points`] **Upload your Jupyter Notebook, Explanation Document, and Report** to your GitHub repository.
- Ensure the repository is public and contains:
  - `users.csv`, `movies.csv` and `ratings.csv` [These are the Dataframes which were created in part 1. Save and export them as a `.csv` file]
  - `Movie_Recommendation.ipynb`
  - `Pixie_Algorithm_Explanation.pdf` or `.md`
  - `Recommendation_Report.pdf` or `.md`
- **Submit the GitHub repository link in the cell below.**


#### **Example Submission Format**
```text
GitHub Repository: https://github.com/username/Movie-Recommendation
```

In [ ]:
# Submit the Github Link here:


### **Grading Rubric: ITCS 6162 - Data Mining Assignment**


| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **Part 1: Exploring and Cleaning Data (15 pts)**  | Properly loads `u.user`, `u.movies`, and `u.item` datasets into DataFrames | 5 |
|                                           | Handles missing values, duplicates, and inconsistencies appropriately | 5 |
|                                           | Saves the cleaned datasets into CSV files: `users.csv`, `movies.csv`, `ratings.csv` | 5 |
| **Part 2: Collaborative Filtering-Based Recommendation (30 pts)** | Implements user-based collaborative filtering correctly | 10 |
|                                           | Implements item-based collaborative filtering correctly | 10 |
|                                           | Computes similarity measures accurately and provides valid recommendations | 10 |
| **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm) (35 pts)** | Constructs adjacency lists properly from user-movie interactions | 10 |
|                                           | Implements weighted random walk-based recommendation correctly | 15 |
|                                           | Explains and justifies the algorithm design choices (Pixie-inspired) | 10 |
| **Code Quality & Documentation (10 pts)** | Code is well-structured, efficient, and follows best practices | 5 |
|                                           | Markdown explanations and comments are clear and enhance understanding | 5 |
| **Results & Interpretation (5 pts)**      | Provides meaningful insights from the recommendation system's output | 5 |
| **Submission & Report (5 pts)**          | Submits all required files in the correct format (ZIP file with Jupyter notebook, processed CSV files, and project report) | 5 |
| **Total**                                 |                              | 100 |

#### **Bonus (10 pts)**
| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **GitHub Submission**                     | Provides a well-documented GitHub repository with CSV files, a structured README, and a properly formatted Jupyter Notebook | 10 |